In [2]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")

In [26]:
elo_data = pd.read_sql_query(
    "Select DISTINCT * From Elo_AllMatches_Daily_All",
    con=devengine,
)

In [27]:
elo_data=elo_data[elo_data['Sex']=='Womens']

In [28]:
for x in range(1,4):
    for y in range(0,10):
        lower=x+(y*0.1)
        upper=x+((y+1)*0.1)

        test1=elo_data[(elo_data['Elo_Dog_Odds'].ge(lower))&(elo_data['Elo_Dog_Odds'].le(upper))].copy()
        percent=len(test1[test1['Winner']!=test1['Elo_Fav']])/len(test1)
        STAKE=500
        test1['Profit']=test1.apply(lambda x: -STAKE if x['Winner']==x['Elo_Fav'] else (STAKE * x['Elo_Dog_Odds'])-STAKE,axis=1)
        if test1['Profit'].sum() >0 and len(test1) >20:
            print(lower,upper,percent)
            print(test1['Profit'].sum())


1.2 1.3 0.8
50.0
1.3 1.4 0.8723404255319149
4245.0
1.7000000000000002 1.8 0.6404494382022472
5540.0
2.4 2.5 0.42990654205607476
2745.0
2.8 2.9 0.36619718309859156
1450.0
3.0 3.1 0.359375
3030.0
3.1 3.2 0.36507936507936506
4680.0
3.4 3.5 0.3333333333333333
3135.0
3.5 3.6 0.29411764705882354
680.0


In [29]:
for x in range(1,4):
    for y in range(0,10):
        lower=x+(y*0.1)
        upper=x+((y+1)*0.1)

        test1=elo_data[(elo_data['Elo_Fav_Odds'].ge(lower))&(elo_data['Elo_Fav_Odds'].le(upper))].copy()
        if len(test1) > 0:
            percent=len(test1[test1['Winner']==test1['Elo_Fav']])/len(test1)
            STAKE=500
            test1['Profit']=test1.apply(lambda x: -STAKE if x['Winner']!=x['Elo_Fav'] else (STAKE * x['Elo_Fav_Odds'])-STAKE,axis=1)
            if test1['Profit'].sum() >0 and len(test1) >20:
                print(lower,upper,percent)
                print(test1['Profit'].sum())


1.1 1.2 0.8835616438356164
2490.0
1.6 1.7000000000000002 0.6145833333333334
980.0
2.3 2.4 0.4375
545.0


In [3]:
pd.read_sql_query(
    "Select * From Elo_AllMatches_Daily_All_Today --where date < '2022-09-01' and date > '2022-06-31' and (Surface like 'Hard' or Surface like '') --and Elo_Fav_Est_Odds < 1.8",
    con=devengine,
)

,Surface,Date,Sex,Winner,Resulted,Time,Elo_Fav,Elo_Dog,Elo_Fav_Odds,Elo_Dog_Odds,Elo_Fav_Est_Odds,Elo_Dog_Est_Odds,Elo_Fav_Elo,Elo_Dog_Elo,Elo_Fav_Total,Elo_Dog_Total
0,Hard,2022-10-05,Womens,Lucrezia Stefanini,False,21:30,Diane Parry,Lucrezia Stefanini,1.59,2.34,1.808097,2.237475,1592.766562,1555.751978,15,7
1,Clay,2022-10-05,Mens,Andrea Vavassori,False,22:00,Andrea Vavassori,Stefano Travaglia,1.53,2.39,1.237880,5.203801,1693.442225,1443.985365,17,10
2,Hard,2022-10-05,Womens,Veronika Kudermetova,True,04:40,Veronika Kudermetova,Varvara Gracheva,1.09,7.46,1.123518,9.096018,1722.721855,1359.413274,16,12
3,Hard,2022-10-05,Womens,Ons Jabeur,True,03:05,Ons Jabeur,Ann Li,1.13,5.72,1.144131,7.938110,1797.543412,1461.046948,13,4
4,Hard,2022-10-05,Womens,Magdalena Frech,True,00:55,Yasmine Mansouri,Magdalena Frech,12.85,1.03,1.630519,2.585996,1500.000000,1419.879179,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Hard,2022-10-05,Mens,Denis Shapovalov,True,17:00,Denis Shapovalov,Steve Johnson,1.22,4.27,1.305236,4.276149,1715.413387,1509.267945,11,14
69,Hard,2022-10-05,Mens,Miomir Kecmanovic,True,14:30,Yoshihito Nishioka,Miomir Kecmanovic,1.74,2.09,1.205288,5.871209,1793.861090,1518.806367,21,10
70,Hard,2022-10-05,Mens,Soon Woo Kwon,True,12:10,Soon Woo Kwon,Mackenzie Mcdonald,1.65,2.24,1.809564,2.235232,1558.801421,1522.101959,14,14
71,Hard,2022-10-05,Womens,Claire Liu,False,20:00,Katerina Siniakova,Claire Liu,1.58,2.36,1.314301,4.181666,1741.071611,1540.009787,18,13


In [4]:
def Elo(date_six_months_ago, date_today, date_today_formatted):
    data = pd.read_sql_query(
        "Select distinct Surface,Date,Sex,Player_1 as Winner, Player_2 as Loser, Player_1_Odds as Winner_Odds, Player_2_Odds as Loser_Odds FROM AllMatches where (surface like 'Clay' or surface like 'Hard') and tournament not like '%UK Pro%'  and tournament not like '%UTR%' and tournament not like '%Davis%' and date >='{}' and date <= '{}' ".format(
            date_six_months_ago, date_today
        ),
        con=devengine,
    )
    data2 = pd.read_sql_query(
        "Select distinct Surface,Date,Sex,Player_1 as Winner, Player_2 as Loser, Player_1_Odds as Winner_Odds, Player_2_Odds as Loser_Odds,Resulted,Time FROM TodaysMatches where (surface like 'Clay' or surface like 'Hard') and tournament not like '%UK Pro%'  and tournament not like '%UTR%' and tournament not like '%Davis%' ",
        con=devengine,
    )
    data = pd.concat([data, data2])
    data = data.sort_values("Date")  # sort data frame by date
    data["Surface"].str.replace("'b", "")  # drop the b' prefix from the Surface column
    data["Winner"] = data[
        "Winner"
    ].str.strip()  # remove leading and trailing whitespaces from names
    data["Loser"] = data["Loser"].str.strip()
    data = data.reset_index(drop=True)

    def get_elo_rankings(data):
        """
        Function that given the list on matches in chronological order, for each match, computes
        the elo ranking of the 2 players at the beginning of the match.

        Parameters: data(pandas DataFrame) - DataFrame that contains needed information on tennis matches, e.g players names,
        winners, losesrs , surfaces etc

        Return: elo_ranking(pandas DataFrame) - DataFrame that contains the calculated Elo Ratings and the Pwin.

        """
        players = list(
            pd.Series(list(data.Winner) + list(data.Loser)).value_counts().index
        )  # create list of all players
        elo = pd.Series(
            np.ones(len(players)) * 1500, index=players
        )  # create series with initialised elo rating for all players
        matches_played = pd.Series(
            np.zeros(len(players)), index=players
        )  # create series with players' matches initialised at 0 and updated after each match
        ranking_elo = [(1500, 1500)]  # create initial elo's list
        for i in range(1, len(data)):
            w = data.iloc[i - 1, :].Winner  # identify winning player
            l = data.iloc[i - 1, :].Loser  # identify losing player
            elow = elo[w]
            elol = elo[l]
            matches_played_w = matches_played[w]
            matches_played_l = matches_played[l]
            pwin = 1 / (
                1 + 10 ** ((elol - elow) / 400)
            )  # compute prob of winner to win
            K_win = 250 / ((matches_played_w + 5) ** 0.4)  # K-factor of winning player
            K_los = 250 / ((matches_played_l + 5) ** 0.4)  # K-factor of losing player
            new_elow = elow + K_win * (1 - pwin)  # winning player new elo
            new_elol = elol - K_los * (1 - pwin)  # losing player new elo
            elo[w] = new_elow
            elo[l] = new_elol
            matches_played[w] += 1  # update total matches of players
            matches_played[l] += 1
            ranking_elo.append(
                (elo[data.iloc[i, :].Winner], elo[data.iloc[i, :].Loser])
            )

        ranking_elo = pd.DataFrame(ranking_elo, columns=["Elo_Winner", "Elo_Loser"])
        ranking_elo["Prob_Elo"] = 1 / (
            1 + 10 ** ((ranking_elo["Elo_Loser"] - ranking_elo["Elo_Winner"]) / 400)
        )
        ranking_elo["Prob_Elo_Loser"] = 1 / (
            1 + 10 ** ((ranking_elo["Elo_Winner"] - ranking_elo["Elo_Loser"]) / 400)
        )
        return ranking_elo

    elo_rankings = get_elo_rankings(data)
    data = pd.concat([data, elo_rankings], axis=1)

    def get_prob(a):
        """Function that convert decimal odds to probabilities.
        Parameters: a - decimal odd (float)
        Return: a - probability (float)
        """
        a = 1 / a
        return a

    data["Elo_Fav"] = data.apply(
        lambda x: x["Winner"] if x["Elo_Winner"] > x["Elo_Loser"] else x["Loser"],
        axis=1,
    )
    data["Elo_Dog"] = data.apply(
        lambda x: x["Winner"] if x["Elo_Winner"] < x["Elo_Loser"] else x["Loser"],
        axis=1,
    )
    data["Elo_Fav_Odds"] = data.apply(
        lambda x: x["Winner_Odds"]
        if x["Elo_Winner"] > x["Elo_Loser"]
        else x["Loser_Odds"],
        axis=1,
    )
    data["Elo_Dog_Odds"] = data.apply(
        lambda x: x["Loser_Odds"]
        if x["Elo_Fav_Odds"] == x["Winner_Odds"]
        else x["Winner_Odds"],
        axis=1,
    )
    data["Elo_Fav_Est_Odds"] = data.apply(
        lambda x: 1 / x["Prob_Elo"]
        if x["Elo_Fav_Odds"] == x["Winner_Odds"]
        else 1 / x["Prob_Elo_Loser"],
        axis=1,
    )
    data["Elo_Dog_Est_Odds"] = data.apply(
        lambda x: 1 / x["Prob_Elo_Loser"]
        if x["Elo_Fav_Odds"] == x["Winner_Odds"]
        else 1 / x["Prob_Elo"],
        axis=1,
    )
    tomorrow = datetime.datetime.now() + datetime.timedelta(days=0)
    current_date = tomorrow.strftime("%Y-%m-%d")
    data[
        ["Elo_Fav_Odds", "Elo_Dog_Odds", "Elo_Fav_Est_Odds", "Elo_Dog_Est_Odds"]
    ] = data[
        ["Elo_Fav_Odds", "Elo_Dog_Odds", "Elo_Fav_Est_Odds", "Elo_Dog_Est_Odds"]
    ].astype(
        "float"
    )
    data["Wins"] = data.groupby("Winner").cumcount() + 1
    data["Losses"] = data.groupby("Loser").cumcount() + 1
    data2 = data.copy(deep=True)
    dataloser = data.copy(deep=True)
    data2["Winner"] = data2["Loser"]
    data3 = pd.concat([data, data2]).sort_values("Date")
    data3.reset_index(drop=True, inplace=True)
    data3["WinnerTotal"] = data3.groupby("Winner").cumcount() + 1
    data3 = data3[pd.notnull(data3["Surface"])]
    data4 = data3.merge(
        data,
        how="inner",
        left_on=["Date", "Winner", "Loser"],
        right_on=["Date", "Winner", "Loser"],
    )
    dataloser["Loser"] = dataloser["Winner"]
    data9 = pd.concat([data, dataloser]).sort_values("Date")
    data9.reset_index(drop=True, inplace=True)
    data9["LoserTotal"] = data9.groupby("Loser").cumcount() + 1
    data = data9.merge(
        data4,
        how="inner",
        left_on=["Date", "Winner", "Loser"],
        right_on=["Date", "Winner", "Loser"],
    )

    # data1 = data[data["Date"] != current_date]
    data1 = data
    data = data1
    data = data[data.columns.drop(list(data.filter(regex="_y")))]
    data = data[data.columns.drop(list(data.filter(regex="_x")))]
    data = data.drop(
        columns=["Winner_Odds", "Loser_Odds", "Prob_Elo", "Prob_Elo_Loser", "Loser"],
        axis=1,
    )
    data["Elo_Fav_Elo"] = data.apply(
        lambda x: x["Elo_Winner"] if x["Winner"] == x["Elo_Fav"] else x["Elo_Loser"],
        axis=1,
    )
    data["Elo_Dog_Elo"] = data.apply(
        lambda x: x["Elo_Winner"] if x["Winner"] != x["Elo_Fav"] else x["Elo_Loser"],
        axis=1,
    )
    data["Elo_Fav_Total"] = data.apply(
        lambda x: x["WinnerTotal"] if x["Winner"] == x["Elo_Fav"] else x["LoserTotal"],
        axis=1,
    )
    data["Elo_Dog_Total"] = data.apply(
        lambda x: x["WinnerTotal"] if x["Winner"] != x["Elo_Fav"] else x["LoserTotal"],
        axis=1,
    )
    data = data.drop(
        columns=[
            "Wins",
            "Losses",
            "LoserTotal",
            "WinnerTotal",
            "Elo_Loser",
            "Elo_Winner",
        ],
        axis=1,
    )
    data = data[data["Date"].str.contains((date_today_formatted))]
    data.to_sql(
        "Elo_AllMatches_Daily_All_Today",
        con=devengine,
        if_exists="replace",
        index=False,
    )
    # data2 = data[data["Date"] == current_date]
    # data2.to_sql("Elo_AllMatches_Today", con=devengine, if_exists="replace", index=False)
    # playsound(r"C:\Users\chris\Music\beep-09.mp3")



In [6]:
dog_df = pd.DataFrame()
fav_df = pd.DataFrame()
import datetime
from dateutil.relativedelta import *


def get_data(month_year, past, query, fav_dog, id):
    global dog_df, fav_df
    date_today = datetime.datetime.now() + relativedelta(days=0)
    date_today_formatted = date_today.strftime("%Y-%m-%d")
    if month_year == "month":
        date_six_months_ago = date_today + relativedelta(months=-past)
    else:
        date_six_months_ago = date_today + relativedelta(years=-past)
    # print(date_six_months_ago)
    Elo(date_six_months_ago, date_today, date_today_formatted)

    query_result = pd.read_sql_query(
        query,
        con=devengine,
    )
    if query_result.empty == False:
        query_result["Id"] = id
        if fav_dog == "fav":
            fav_df = pd.concat([fav_df, query_result])
        else:
            dog_df = pd.concat([dog_df, query_result])


def all():
    # Mens 2 years fav
    month_year = "year"
    past = 2
    fav_dog = "fav"
    id = "2yearsfav"
    query = "Select * From Elo_AllMatches_Daily_All_Today where ((Elo_Fav_Odds BETWEEN 2 AND 2.1))\
        and sex = 'Mens'"
    get_data(month_year, past, query, fav_dog, id)

    # Mens 2 years dog
    month_year = "year"
    past = 2
    fav_dog = "dog"
    id = "2yearsdog"
    query = "Select * From Elo_AllMatches_Daily_All_Today where ( \
        (Elo_Dog_Odds BETWEEN 3.6 AND 3.7) OR (Elo_Dog_Odds BETWEEN 1.2 AND 1.3)) and sex = 'Mens'"
    get_data(month_year, past, query, fav_dog, id)

    # Mens 6 months fav
    month_year = "month"
    past = 6
    fav_dog = "fav"
    id = "6monthsfav"
    query = "Select * From Elo_AllMatches_Daily_All_Today where ((Elo_Fav_Odds BETWEEN 2 AND 2.1) OR (Elo_Fav_Odds BETWEEN 3.5 AND 3.6)) \
        and sex = 'Mens'"
    get_data(month_year, past, query, fav_dog, id)

    # Mens 6 months dog
    month_year = "month"
    past = 6
    fav_dog = "dog"
    id = "6monthsdog"
    query = "Select * From Elo_AllMatches_Daily_All_Today where (\
        (Elo_Dog_Odds BETWEEN 3.6 AND 3.7)) and sex = 'Mens'"
    get_data(month_year, past, query, fav_dog, id)

    # Mens 3 months fav
    month_year = "month"
    past = 3
    fav_dog = "fav"
    id = "3monthsfav"
    query = "Select * From Elo_AllMatches_Daily_All_Today where ((Elo_Fav_Odds BETWEEN 2 AND 2.1) OR (Elo_Fav_Odds BETWEEN 2.2 AND 2.3) \
        OR (Elo_Fav_Odds BETWEEN 2.5 AND 2.6) OR (Elo_Fav_Odds BETWEEN 2.9 AND 3.1) OR (Elo_Fav_Odds BETWEEN 3.5 AND 3.7)) and sex = 'Mens'"
    get_data(month_year, past, query, fav_dog, id)

    # Mens 3 months dog
    month_year = "month"
    past = 3
    fav_dog = "dog"
    id = "3monthsdog"
    query = "Select * From Elo_AllMatches_Daily_All_Today where ((Elo_Dog_Odds BETWEEN 3.1 AND 3.3) OR (Elo_Dog_Odds BETWEEN 3.6 AND 3.7) OR (Elo_Dog_Odds BETWEEN 1.1 AND 1.2) \
        OR (Elo_Dog_Odds BETWEEN 3.8 AND 3.9)) and sex = 'Mens'"
    get_data(month_year, past, query, fav_dog, id)

    ##################

    # Womens 2 years dog
    month_year = "year"
    past = 2
    fav_dog = "dog"
    id = "2yearsdog"
    query = "Select * From Elo_AllMatches_Daily_All_Today where (  \
        (Elo_Dog_Odds BETWEEN 3.4 AND 3.5) OR (Elo_Dog_Odds BETWEEN 3 AND 3.2) OR (Elo_Dog_Odds BETWEEN 1.2 AND 1.4)) and sex = 'Womens'"
    get_data(month_year, past, query, fav_dog, id)

    # Womens 2 years fav
    month_year = "year"
    past = 2
    fav_dog = "fav"
    id = "2yearsdog"
    query = "Select * From Elo_AllMatches_Daily_All_Today where (  \
        (Elo_Fav_Odds BETWEEN 1.1 AND 1.2)) and sex = 'Womens'"
    get_data(month_year, past, query, fav_dog, id)

    # Womens 6 months dog
    month_year = "month"
    past = 6
    fav_dog = "dog"
    id = "6monthsdog"
    query = "Select * From Elo_AllMatches_Daily_All_Today where ((Elo_Dog_Odds BETWEEN 3.1 AND 3.3) OR (Elo_Dog_Odds BETWEEN 3.5 AND 3.6) OR (Elo_Dog_Odds BETWEEN 1.3 AND 1.4) \
        ) and sex = 'Womens'"
    get_data(month_year, past, query, fav_dog, id)

    # Womens 3 months dog
    month_year = "month"
    past = 3
    fav_dog = "dog"
    id = "3monthsdog"
    query = "Select * From Elo_AllMatches_Daily_All_Today where ((Elo_Dog_Odds BETWEEN 2.4 AND 2.5) OR (Elo_Dog_Odds BETWEEN 3.1 AND 3.2) OR (Elo_Dog_Odds BETWEEN 1.1 AND 1.2)\
        OR (Elo_Dog_Odds BETWEEN 3.4 AND 3.6)) and sex = 'Womens'"
    get_data(month_year, past, query, fav_dog, id)

    # Womens 3 months fav
    month_year = "month"
    past = 3
    fav_dog = "fav"
    id = "3monthsfav"
    query = "Select * From Elo_AllMatches_Daily_All_Today where ((Elo_Fav_Odds BETWEEN 1.1 AND 1.2)\
        ) and sex = 'Womens'"
    get_data(month_year, past, query, fav_dog, id)

    # Womens 6 months fav
    month_year = "month"
    past = 6
    fav_dog = "fav"
    id = "6monthsfav"
    query = "Select * From Elo_AllMatches_Daily_All_Today where ((Elo_Fav_Odds BETWEEN 1.1 AND 1.2)\
        ) and sex = 'Womens'"
    get_data(month_year, past, query, fav_dog, id)

    #################

    return fav_df, dog_df

In [7]:
fav_df, dog_df = all()

In [ ]:
fav_df2, dog_df2 = clay()

In [65]:
elo_data_today[((elo_data_today['Elo_Fav_Odds'].ge(2))&(elo_data_today['Elo_Fav_Odds'].le(2.1)))]#|((elo_data_today['Elo_Fav_Odds'].ge(2.2))&(elo_data_today['Elo_Fav_Odds'].le(2.3)))]


,Surface,Date,Sex,Winner,Resulted,Time,Elo_Fav,Elo_Dog,Elo_Fav_Odds,Elo_Dog_Odds,Elo_Fav_Est_Odds,Elo_Dog_Est_Odds,Elo_Fav_Elo,Elo_Dog_Elo,Elo_Fav_Total,Elo_Dog_Total
20,Hard,2022-09-18,Mens,Facundo Mena,False,09:00,Facundo Mena,Keegan Smith,2.09,1.68,1.422243,3.368307,1668.550615,1518.775433,20,11
31,Hard,2022-09-18,Womens,Chihiro Muramatsu,False,15:00,Chihiro Muramatsu,Rina Saigo,2.03,1.70,1.931995,2.072968,1553.952478,1541.717850,2,2


In [66]:
elo_data_today[((elo_data_today['Elo_Dog_Odds'].ge(2.4))&(elo_data_today['Elo_Dog_Odds'].le(2.5)))]#|((elo_data_today['Elo_Fav_Odds'].ge(2.2))&(elo_data_today['Elo_Fav_Odds'].le(2.3)))]


,Surface,Date,Sex,Winner,Resulted,Time,Elo_Fav,Elo_Dog,Elo_Fav_Odds,Elo_Dog_Odds,Elo_Fav_Est_Odds,Elo_Dog_Est_Odds,Elo_Fav_Elo,Elo_Dog_Elo,Elo_Fav_Total,Elo_Dog_Total
30,Hard,2022-09-18,Womens,Nadia Podoroska,True,23:40,Linda Fruhvirtova,Nadia Podoroska,1.57,2.41,1.784323,2.274984,1640.223164,1598.021253,10,10
